# [Access to GMrepo using python through RESTful APIs](https://github.com/evolgeniusteam/GMrepoProgrammableAccess/blob/master/programmable-access/python.md)

In [1]:
import requests
import json
from pandas.core.frame import DataFrame
import pandas as pd
import numpy as np

## Phenotypes
Get all phenotypes and statistics

In [121]:
### -- get all phenotypes  --
url = 'https://gmrepo.humangut.info/api/get_all_phenotypes'
pheno_01 = requests.post(url, data={})
pheno_01_cont = pheno_01.json().get('phenotypes')

## -- a DateFrame --
phenotypes = DataFrame(pheno_01_cont)
phenotypes.head()

 # MeSH ID  Vs disease

In [123]:
phenotypes[['disease','term','note']].to_csv('meshID_phenotypes.csv') #MeSH ID


In [3]:
import pandas as pd
diseasef='./meshID_phenotypes.csv'
disease = pd.read_csv(diseasef,comment='#',index_col=0)
disease.head()
#疾病名字太长，用meshid

,disease,term,note
0,C537163,Pediatric Autoimmune Neuropsychiatric Disorder...,OBSESSIVE-COMPULSIVE DISORDER and TIC DISORDER...
1,D000067011,Severe Acute Malnutrition,Acute form of MALNUTRITION which usually affec...
2,D000067877,Autism Spectrum Disorder,Wide continuum of associated cognitive and neu...
3,D000236,Adenoma,A benign epithelial tumor with a glandular org...
4,D000544,Alzheimer Disease,A degenerative disease of the BRAIN characteri...


In [4]:
def replaceMany(orig_str, substring_list, new_str):
    # Traverse the substring list to replace
    for string in substring_list :
        # Test if string exists in the original string
        if string in orig_str :
            # Replace the string
            orig_str = orig_str.replace(string, new_str)
    return orig_str

disease.term  = [replaceMany(s,[' ',',','__'] ,'_') for s in disease.term]

In [5]:
disease.head()

,disease,term,note
0,C537163,Pediatric_Autoimmune_Neuropsychiatric_Disorder...,OBSESSIVE-COMPULSIVE DISORDER and TIC DISORDER...
1,D000067011,Severe_Acute_Malnutrition,Acute form of MALNUTRITION which usually affec...
2,D000067877,Autism_Spectrum_Disorder,Wide continuum of associated cognitive and neu...
3,D000236,Adenoma,A benign epithelial tumor with a glandular org...
4,D000544,Alzheimer_Disease,A degenerative disease of the BRAIN characteri...


In [31]:
# disease
disease.to_csv('meshID_phenotypes_.csv',index=False) 

# mkdirs

In [94]:
seq = ['metagenome','amplicon']
tax = ['genus', 'species']

In [95]:
outdir ='./' 
from pathlib import Path
repo = Path(outdir)

In [7]:
mdiseasef='./metagenome_otutable_diseaseIDs.tsv'
mdisease = pd.read_csv(mdiseasef,comment='#',index_col=0,header=None,sep='\s')
mdisease.head()
#疾病名字太长，还是用meshid
# metadata不修改了

,1
0,
1,D009765
2,D009767
3,D006973
4,D007234
5,D000755


In [97]:
for id in mdisease.iloc[:,0].values:
    '''有的疾病没有metagenome数据，不需要mkdir，有的没有amplicom数据不需要mkdir'''
    term = disease[disease['disease']==id]['term'].values[0]
    dis_dir = id + '_' + term.replace(' ','_')[:20]
    path = repo.joinpath(seq[0],'diseases',dis_dir,tax[1])
    if not path.is_dir(): 
        path.mkdir(exist_ok=False,parents=True)

In [ ]:
adiseasef='./Amplicon_otutable_diseaseIDs.tsv'
adisease = pd.read_csv(adiseasef,comment='#',index_col=0,sep='\s',header=None)
#疾病名字太长，还是用meshid
# metadata不修改了

In [107]:
for id in adisease.iloc[:,0].values:
    '''有的疾病没有metagenome数据，不需要mkdir，有的没有amplicom数据不需要mkdir'''
    term = disease[disease['disease']==id]['term'].values[0]
    dis_dir = id + '_' + term.replace(' ','_')[:20]
    path = repo.joinpath(seq[1],'diseases',dis_dir,tax[0])
    if not path.is_dir(): 
        path.mkdir(exist_ok=False,parents=True)

# map file for each sample/runid

download metadata for all metagenome samples

https://gmrepo.humangut.info/home

Query logic: ( experiment_type = 'Metagenomics' AND QCStatus = 1 )

user_selected_run_list_all_disease_metagenomics_goodruns.txt

In [5]:
all_meta_samples='./metagenome/user_selected_run_list_all_disease_metagenomics_goodruns.txt'
all_meta_samples_ids = pd.read_csv(all_meta_samples,sep='\t',comment='#')
all_meta_samples_ids.head()

,Run ID,Project ID,Experiment type,Nr. reads sequenced,Country,Sex,Host age,BMI,Recent antibiotics use,Disease MESH ID,Disease name,QC status,Nr. associated phenotypes
0,ERR011089,PRJEB2054,Metagenomics,4190512.0,Denmark,Female,59.0,NaN,NaN,D006262,1,1,NaN
1,ERR011090,PRJEB2054,Metagenomics,11169067.0,Denmark,Female,59.0,NaN,NaN,D006262,1,1,NaN
2,ERR011091,PRJEB2054,Metagenomics,7928489.0,Denmark,Female,59.0,NaN,NaN,D006262,1,1,NaN
3,ERR011092,PRJEB2054,Metagenomics,4478452.0,Denmark,Male,69.0,NaN,NaN,D006262,1,1,NaN
4,ERR011093,PRJEB2054,Metagenomics,10857325.0,Denmark,Male,69.0,NaN,NaN,D006262,1,1,NaN


In [72]:
all_samples_metagenome_metadata = all_meta_samples_ids[['Run ID','Project ID','Country','Sex','Host age','BMI','Recent antibiotics use','Disease MESH ID']].drop_duplicates() #samples with metadata imformation

In [75]:
meta_mf=all_samples_metagenome_metadata.set_index('Run ID').loc[species_df.RunID.unique(),:]# #interset with genus samples

In [76]:
meta_samples = meta_mf.index.unique().values

In [81]:
meta_mf.shape

(12109, 7)

# Get relative species/genus abundances in samples/runs associated with a phenotype

To get the related information, two input parameters are required:

MeSH ID of interests, e.g. D003093 for Colitis, Ulcerative

In [97]:
url = 'https://gmrepo.humangut.info/api/getRunDetailsByRunID'
all_run_ids = meta_samples

def getRunDetailsByRunID(url, all_run_ids,outdir):
    species_df =  DataFrame()
    genus_df = DataFrame()

    for id in all_run_ids:
        query = {"run_id":id} 
        data = requests.post(url, data=json.dumps(query)).json()

        ## --get DataFrames
        species = DataFrame(data.get("species"))
        species["RunID"] = id
        species_df = pd.concat([species_df, species], axis=0, ignore_index=True)
        
        genus = DataFrame(data.get("genus"))
        genus["RunID"] = id
        genus_df = pd.concat([genus_df, genus], axis=0, ignore_index=True)


    # all species with amplicon species
    species_df.to_csv('./{}/all_sample_species_relative_abundance.csv'.format(outdir), index=False)
    genus_df.to_csv('./{}/all_sample_genus_relative_abundance.csv'.format(outdir), index=False)
    return species_df,  genus_df

## metagenome

In [12]:
import pandas as pd
species_df = pd.read_csv('./metagenome/all_sample_species_relative_abundance.csv')
genus_df =  pd.read_csv('./metagenome/all_sample_genus_relative_abundance.csv') 



### Delete unkown (-1) and Others (0) for they are not meaningful features, drop_duplicates


In [13]:
sdf = species_df[species_df.ncbi_taxon_id>0].drop_duplicates()
gdf = genus_df[genus_df.ncbi_taxon_id>0].drop_duplicates()  #del unknown and Others

### species table

In [14]:
# sdf=sdf[sdf.RunID.isin(meta_samples)]
sdf=sdf.groupby(['ncbi_taxon_id','RunID','scientific_name']).sum().reset_index()
sdf_table = sdf.pivot(index='ncbi_taxon_id', columns='RunID', values='relative_abundance')
sdf_table.fillna(value=0).to_csv('species_table.csv')

In [85]:
sdf[sdf.RunID=='ERR2242021']

,ncbi_taxon_id,RunID,scientific_name,relative_abundance
256,329,ERR2242021,Ralstonia pickettii,0.11139
1961,562,ERR2242021,Escherichia coli,0.14762
25786,1254,ERR2242021,Pediococcus acidilactici,0.18238
29421,1352,ERR2242021,Enterococcus faecium,99.26710


## genus table

In [86]:
gdf=gdf.groupby(['ncbi_taxon_id','RunID','scientific_name']).sum().reset_index()
gdf_table = gdf.pivot(index='ncbi_taxon_id', columns='RunID', values='relative_abundance')
gdf_table.fillna(value=0).to_csv('genus_table.csv')

## get taxonomy file

In [90]:
species_tax = sdf[['ncbi_taxon_id','scientific_name']].drop_duplicates()
species_tax.to_csv('species_tax.csv',index=False)

In [91]:
genus_tax = gdf[['ncbi_taxon_id','scientific_name']].drop_duplicates()
genus_tax.to_csv('genus_tax.csv',index=False)

# Amplicon

download metadata for all amplicon samples

user_selected_run_list_PyOfHsgnu7.txt

In [ ]:
def get_pivot_table(species_df,outdir='amplicon', tax='species'):  
    sdf = species_df[species_df.ncbi_taxon_id>0].drop_duplicates() #del unknown and Others
    sdf=sdf.groupby(['ncbi_taxon_id','RunID','scientific_name']).sum().reset_index() #sum duplicate rows
    sdf_table = sdf.pivot(index='ncbi_taxon_id', columns='RunID', values='relative_abundance') # pivot table
    sdf_table.fillna(value=0).to_csv('./{}/{}_table.csv'.format(outdir, tax)) # save    
    
    species_tax = sdf[['ncbi_taxon_id','scientific_name']].drop_duplicates()
    species_tax.to_csv('./{}/{}_tax.csv'.format(outdir, tax),index=False)
    return sdf, species_tax

In [105]:
all_amplicon_samples='./amplicon/user_selected_run_list_PyOfHsgnu7.txt'

def get_metadata(samplefile):    
    all_samples_ids = pd.read_csv(samplefile,sep='\t',comment='#')
    all_samples_metadata = all_samples_ids[['Run ID','Project ID','Country','Sex','Host age','BMI','Recent antibiotics use','Disease MESH ID']].drop_duplicates() #samples with metadata imformation
    samples = all_samples_metadata['Run ID'].unique()
    return samples, all_samples_metadata

In [106]:
amplicon_samples , amplicon_samples_metadata = get_metadata(all_amplicon_samples)

In [111]:
url = 'https://gmrepo.humangut.info/api/getRunDetailsByRunID'
all_run_ids = amplicon_samples
outdir = 'amplicon'
amplicon_all_species_df,  amplicon_all_genus_df = getRunDetailsByRunID(url, all_run_ids,outdir)

amplicon_species_df, amplicon_species_tax = get_pivot_table(amplicon_all_species_df,outdir='amplicon', tax='species')
amplicon_genus_df, amplicon_genus_tax = get_pivot_table(amplicon_all_genus_df,outdir='amplicon', tax='genus')


In [117]:
amplicon_all_species_df.head()

,ncbi_taxon_id,taxon_rank_level,relative_abundance,scientific_name,RunID
0,1515,species,6.79631,Ruminiclostridium thermocellum,DRR049319
1,59310,species,2.43579,Streptococcus macedonicus,DRR049319
2,589436,species,1.80195,Prevotella fusca,DRR049319
3,29389,species,1.63934,Streptococcus alactolyticus,DRR049319
4,102684,species,1.56966,Streptococcus infantarius,DRR049319


In [109]:
len(amplicon_samples)


16679

In [112]:
amplicon_samples_metadata.shape

(19495, 8)

In [114]:
amplicon_species_tax.shape

(1311, 2)

# get NCBI taxonomic lineage and tree

In [ ]:
import os
import gneiss
from ete3 import NCBITaxa
ncbi = NCBITaxa()
import pandas as pd


def get_tax(ncbi_tax_id,rank='g'):
    '''rank<- [k,p,c,o,f,g,s]'''
    lineage = ncbi.get_lineage(ncbi_tax_id)
    names = ncbi.get_taxid_translator(lineage)
    ranks = ['k','p','c','o','f','g','s']   
    taxs = [names[taxid] for taxid in lineage][2:] 
    if len(taxs)==6:
        rank_tax = ';'.join(map(lambda x: x[0]+ '__'+ x[1], zip(ranks, taxs))) +';s__'
    else:
        rank_tax = ';'.join(map(lambda x: x[0]+ '__'+ x[1], zip(ranks, taxs)))
    return rank_tax

def get_tax_tree_from_ncbiID(
    taxf=None,
    rank='g',
    outdir='./',
):
    tax_ori = pd.read_csv(taxf, index_col=0)
    tax = [get_tax(id,rank=rank) for id in tax_ori.index]
    tax_ori.iloc[:,0] = tax
    tax_ori.to_csv(outdir + '{}_tax.xls'.format(rank), sep='\t')   # for qiime import convert tsv
    tax_ori.to_csv(outdir + '{}_tax_nohead.xls'.format(rank), sep='\t',  header=False)
    tree = ncbi.get_topology(tax_ori.index, collapse_subspecies=True)
    #intermediate_nodes=False, rank_limit=None, collapse_subspecies=False, annotate=True
    tree.write(features=["sci_name", "rank"], format=9, outfile=outdir + "{}_tree9.nwk".format(rank))


In [ ]:
os.chdir('/data/database/GMrepo/')
os.getcwd()

taxfg='/data/database/GMrepo/amplicon/genus_tax.csv'
rankg = 'g'

outdir = '/data/database/GMrepo/amplicon/'

taxfs= '/data/database/GMrepo/amplicon/species_tax.csv'
ranks = 's'
# tree.write(format=9, outfile=outdir + "{}_tree91.nwk".format(rank))

for taxf,rank in zip([taxfg,taxfs],['g', 's']):
    get_tax_tree_from_ncbiID(
        taxf=taxf,
        rank=rank,
        outdir=outdir
    )
    


## seperate phenotypes and generate phyloseq.Rdata

### notebook: [phyloseq.ipynb](./phyloseq.ipynb)

phyloseq.ipynb 生成每个疾病的otutable sample_data，taxtable，tree以及phyloseq.Rdata对象文件。